In [1]:
!pip install ta vectorbt

# Veri & Kütüphaneler

In [2]:
import yfinance as yf
import vectorbt as vbt
import pandas as pd
import itertools
import ta
import numpy as np

In [3]:
symbol = 'USDTRY=X'
data = yf.download(symbol)

close = data['Close']

[*********************100%%**********************]  1 of 1 completed


# MA

In [4]:
ma3 = close.rolling(window=3).mean()
ma5 = close.rolling(window=5).mean()
ma10 = close.rolling(window=10).mean()
ma20 = close.rolling(window=20).mean()
ma30 = close.rolling(window=30).mean()
ma50 = close.rolling(window=50).mean()
ma100 = close.rolling(window=100).mean()
ma200 = close.rolling(window=200).mean()

entries_exits = [
    ('MA20 > MA50', ma20 > ma50, 'MA20 < MA50', ma20 < ma50),
    ('MA50 > MA200', ma50 > ma200, 'MA50 < MA200', ma50 < ma200),
    ('MA50 < MA200', ma50 < ma200, 'MA50 > MA200', ma50 > ma200),
    ('MA3 < MA5', ma3 < ma5, 'MA3 > MA5', ma3 > ma5),
    ('MA5 < MA10', ma5 < ma10, 'MA5 > MA10', ma5 > ma10),
    ('MA10 < MA20', ma10 < ma20, 'MA10 > MA20', ma10 > ma20),
    ('MA30 < MA50', ma30 < ma50, 'MA30 > MA50', ma30 > ma50),
    ('MA50 < MA100', ma50 < ma100, 'MA50 > MA100', ma50 > ma100),
]

# RSI

In [5]:
rsi_indicator = ta.momentum.RSIIndicator(close, window=14)
rsi = rsi_indicator.rsi()
entries_exits.append(('RSI < 30', rsi < 30, 'RSI > 70', rsi > 70))

# RSI + MA

In [6]:
entries_exits.append(('MA20 > MA50 & RSI < 30', (ma20 > ma50) & (rsi < 30), 'MA20 < MA50 | RSI > 70', (ma20 < ma50) | (rsi > 70)))

# MACD

In [7]:
macd_indicator = ta.trend.MACD(close)
macd = macd_indicator.macd()
signal = macd_indicator.macd_signal()
entries_exits.append(('MACD > Signal', macd > signal, 'MACD < Signal', macd < signal))

# MACD + RSI

In [8]:
entries_exits.append(('MACD > Signal & RSI < 30', (macd > signal) & (rsi < 30), 'MACD < Signal & RSI > 70', (macd < signal) & (rsi > 70)))

# Bollinger Bandı

In [9]:
bb_indicator = ta.volatility.BollingerBands(close, window=20, window_dev=2)
bb_high = bb_indicator.bollinger_hband()
bb_low = bb_indicator.bollinger_lband()
entries_exits.append(('Close < BB Low', close < bb_low, 'Close > BB High', close > bb_high))

# Ichimoku Bulutu

In [10]:
ichimoku = ta.trend.IchimokuIndicator(data['High'], data['Low'], window1=9, window2=26, window3=52)
ichimoku_a = ichimoku.ichimoku_a()
ichimoku_b = ichimoku.ichimoku_b()
entries_exits.append(('Close > Ichimoku A', close > ichimoku_a, 'Close < Ichimoku B', close < ichimoku_b))

# Stokastik Osilatör

In [11]:
stoch_k = ta.momentum.stoch(close, data['Low'], data['High'], window=14, smooth_window=3)
stoch_d = stoch_k.rolling(window=3).mean()
entries_exits.append(('Stoch K < 20', stoch_k < 20, 'Stoch K > 80', stoch_k > 80))
entries_exits.append(('Stoch K > Stoch D', stoch_k > stoch_d, 'Stoch K < Stoch D', stoch_k < stoch_d))

# Parabolik SAR

In [12]:
psar = ta.trend.PSARIndicator(data['High'], data['Low'], close, step=0.02, max_step=0.2)
entries_exits.append(('Close > PSAR', close > psar.psar(), 'Close < PSAR', close < psar.psar()))

# Etmenler

In [13]:
init_cash = 1000
stop_loss_values = np.arange(0.01,0.20,0.01)
take_profit_values = np.arange(0.1, 1, 0.1)

# Kombinasyonların Denenmesi

In [14]:
best_performance = None
best_stats = None
best_entries = None
best_exits = None
best_entries_name = None
best_exits_name = None
best_stop_loss = None
best_take_profit = None

for entry_name, entries, exit_name, exits in entries_exits:
    for sl in stop_loss_values:
        for tp in take_profit_values:
            pf = vbt.Portfolio.from_signals(
                close, entries, exits, init_cash=init_cash, sl_stop=sl, tp_stop=tp)
            performance = pf.total_return()

            if best_performance is None or performance > best_performance:
                best_performance = performance
                best_stats = pf.stats()
                best_entries = entries
                best_exits = exits
                best_entries_name = entry_name
                best_exits_name = exit_name
                best_stop_loss = sl
                best_take_profit = tp

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requir

# İstatistikler

In [15]:
print("En iyi performansı sağlayan strateji:\n")
print("Giriş Sinyali:", best_entries_name)
print("Çıkış Sinyali:", best_exits_name)
print("\nStop-Loss Değeri:", best_stop_loss)
print("Take-Profit Değeri:", best_take_profit, "\n")
print(best_stats)

pf = vbt.Portfolio.from_signals(
    close, best_entries, best_exits, init_cash=init_cash, sl_stop=best_stop_loss, tp_stop=best_take_profit)

En iyi performansı sağlayan strateji:

Giriş Sinyali: Stoch K > Stoch D
Çıkış Sinyali: Stoch K < Stoch D

Stop-Loss Değeri: 0.04
Take-Profit Değeri: 0.1 

Start                         2005-01-03 00:00:00
End                           2024-06-04 00:00:00
Period                                       5052
Start Value                                1000.0
End Value                           428902.856252
Total Return [%]                     42790.285625
Benchmark Return [%]                  2311.741051
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        11.074857
Max Drawdown Duration                       439.0
Total Trades                                  981
Total Closed Trades                           980
Total Open Trades                               1
Open Trade PnL                                0.0
Win Rate [%]                            64.693878
Best Trade [%]                          28.42